In [1]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import sqlite3
import mysql.connector as cnt
import random
# import setuptools.dist
import undetected_chromedriver as uc
import warnings
warnings.filterwarnings('ignore')

In [2]:
def getColNamesList(table, dbname):
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''select * from {table} limit 0'''.format(table=table)
    s = c.execute(sql)
    # r = s.description
    column_names = [description[0] for description in c.description]
    return column_names

def alterTableAddColumn(cols_i_want_to_have, cols_i_have, table, base):
    db = sqlite3.connect(base)
    c = db.cursor()
    sql = '''alter table {tname}
    add column `{n}`
    '''
    for cl in cols_i_want_to_have:
        if cl not in cols_i_have:
            c.execute(sql.format(tname=table, n=cl))
            time.sleep(1)
            db.commit()
            time.sleep(1)
    c.close()
    db.close()
    return True


def createMainTable(dbname:str) -> bool:
    sql = '''
    drop table if exists DviraciaiPigu
    '''
    db = sqlite3.connect(dbname)
    c = db.cursor()
    c.execute(sql)
    sql = '''create table if not exists DviraciaiPigu
    (
        nuoroda text,
        firstInfo text
    )
    '''
    c.execute(sql)
    c.close()
    db.close()
    return True
    pass

def writeMany(lst, table, dbname) -> bool:
    db = sqlite3.connect(dbname)
    c = db.cursor()
    sql = '''insert into {table} values ({n})'''
    q = ['?' for i in range(0,len(lst))]
    q_ = ', '.join(q)
    r_ = sql.format(table=table, n=q_)
    # print('::PROVIDED::', lst, len(lst), '::SQL COMMAND::', r_, len(q), sep='\n')
    c.execute(r_, lst)
    db.commit()
    c.close()
    db.close()
    return True


In [3]:
opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
url = 'https://pigu.lt/lt/dviraciai?page=1'
# url = 'https://pigu.lt/lt/saldytuvai/'
driver.get(url)
time.sleep(5)
source = driver.page_source


driver.close()
print('Baigta!!!')

Baigta!!!


In [4]:
bs = BeautifulSoup(source, 'html.parser')
p_nuorodos = []

linkai = bs.find_all('div', {'class':'c-product-card__title'})
for linkas in linkai:
    link = linkas.find('a')['href']
    # print(link)
    p_nuorodos.append(link)

print(len(p_nuorodos))
print(p_nuorodos)


max_page_number = bs.find('a', {'class':'pageSlot-7'}).text
print(max_page_number)



60
['https://pigu.lt/lt/dviraciai/kalnu-dviratis-louke-fat-bike-x-treme?id=69190496', 'https://pigu.lt/lt/dviraciai/dviratis-goetze-28-6sp-su-pintu-krepsiu?id=67972891', 'https://pigu.lt/lt/dviraciai/vyriskas-kalnu-dviratis-louke-1-26-juodas?id=14804875', 'https://pigu.lt/lt/dviraciai/miesto-dviratis-n1-cruiser-10-28-smelio?id=43672128', 'https://pigu.lt/lt/dviraciai/vaikiskas-dviratis-bimbo-bike-candy-20-baltas?id=29605805', 'https://pigu.lt/lt/dviraciai/dviratis-romet-wagant-2-28-2023-pilkas?id=70421606', 'https://pigu.lt/lt/dviraciai/dviratukas-vaikams-bimbo-bike-14-boy-cosmos?id=29751990', 'https://pigu.lt/lt/dviraciai/moteriskas-dviratis-davi-maria-alu-160-185?id=69969224', 'https://pigu.lt/lt/dviraciai/kalnu-dviratis-mtb-29-juodas?id=88165355', 'https://pigu.lt/lt/dviraciai/kalnu-dviratis-gust-roxter-29-zalias?id=80871715', 'https://pigu.lt/lt/dviraciai/miesto-dviratis-n1-cruiser-10-28-juodas?id=43672553', 'https://pigu.lt/lt/dviraciai/dviratis-capriolo-lc-93-29-raudonas?id=81002

In [7]:
for nuoroda in p_nuorodos[:1]:
    print(nuoroda)
    opcijos = Options()
    opcijos.add_argument('--incognito')
    opcijos.add_argument('--ignore-certificate-errors')
    opcijos.add_argument('--ignore-ssl-errors')
    driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
    url = nuoroda
    driver.get(url)
    time.sleep(5)
    source = driver.page_source


    driver.quit()

https://pigu.lt/lt/dviraciai/kalnu-dviratis-louke-fat-bike-x-treme?id=69190496


In [8]:
bs = BeautifulSoup(source, 'html.parser')
firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
print(firstInfo)

secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
print(secondInfo)

# x = eval(price['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
# dict(x)
# x['meta']['sell_price']

kaina = bs.find('div', {'class':'c-product__media'})
price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
kaina2 = price['dataLayerItem'].get('price', 'nenurodyta')
print(kaina2)

# if 'price' in price['dataLayerItem']:
#     kaina2 = 'nera'
# else:
#     kaina2 = price['dataLayerItem']['price']
# print(kaina2)


# kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
# print(kaina)

specifikacija_keys = []
specifikacija_values = []
specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
for trow in specifikacija:
    td = trow.find_all('td')
    key = td[0].text.strip()
    value = td[1].text.strip()
    specifikacija_keys.append(key)
    specifikacija_values.append(value)

print(len(specifikacija_keys), specifikacija_keys)
print(len(specifikacija_values), specifikacija_values)




Kalnų dviratis Louke Fat Bike X-treme, 26", juodas
Kalnų dviratis Louke Fat Bike X-treme, 26", juodas
418
24 ['Prekės ID:', 'Kategorija:', 'Prekės pakuočių kiekis:', 'Pakuotės išmatavimai ir svoris (1):', 'Dviračių tipai:', 'Rūšis:', 'Skirta:', 'Prekės ženklas:', 'Rėmo dydis:', 'Svoris:', 'Šakė:', 'Ratlankiai:', 'Galinis permetėjas:', 'Priekinis permetėjas:', 'Pavarų perjungimo rankenėlė:', 'Diskiniai stabdžiai:', 'Stabdžių sistema:', 'Amortizatoriai galiniai:', 'Pardavėjas:', 'Rėmas:', 'Pavarų skaičius:', 'Ratų skersmuo:', 'Rekomenduojamas ūgis:', 'Visos šio ženklo prekės:']
24 ['69190496', 'Dviračiai', '1 vnt.', '1,57 x 0,55 x 0,74 m, 22 kg', 'Kalnų', 'Louke, kalnų dviračiai', 'Vyrams', 'Louke', '18 cm', '16 kg', 'Amortizuojanti', 'Dvigubi aliumininiai', 'L-TWOO\xa0TZ50', 'Nenurodyta', 'L-TWOO EF51-7R/ 3L', 'Taip', 'Diskiniai', 'Yra', 'Pigu.lt, \t\t\t\t\t            JVCB Trade', 'Plieninis', '7', '26 "', '175-185 cm', 'Visos Louke prekės']


In [ ]:
# createMainTable('VarlePigu.db')

True

In [6]:
# Dviraciai Pigu scraper

opcijos = Options()
opcijos.add_argument('--incognito')
opcijos.add_argument('--ignore-certificate-errors')
opcijos.add_argument('--ignore-ssl-errors')
driver = uc.Chrome(use_subprocess=True, suppress_welcome=True, options=opcijos)
# driver = webdriver.Chrome(options=opcijos)
url = 'https://pigu.lt/lt/dviraciai?page=34'
driver.get(url)
time.sleep(5)
source = driver.page_source

bs = BeautifulSoup(source, 'html.parser')
max_page_number = bs.find('a', {'class':'pageSlot-7'}).text
last_page = int(max_page_number)

counts = 1980
psl = 34
# iki_kokio_psl = 29
# while psl < iki_kokio_psl:
while psl < last_page + 1:
    source = driver.page_source
    bs = BeautifulSoup(source, 'html.parser')
    p_nuorodos = []
    linkai = bs.find_all('div', {'class':'c-product-card__title'})
    for linkas in linkai:
        link = linkas.find('a')['href']
        # print(link)
        p_nuorodos.append(link)
        
    for nuoroda in p_nuorodos:
        print(psl, nuoroda)
        url = nuoroda
        driver.get(url)
        time.sleep(random.randint(5,10))
        # time.sleep(4)
        source = driver.page_source
        
        cols_i_have = getColNamesList(table='DviraciaiPigu', dbname='VarlePigu.db')
        
        duomenys = {}
        for col in cols_i_have:
            duomenys[col] = None
            
        duomenys['nuoroda'] = nuoroda
        
        bs = BeautifulSoup(source, 'html.parser')
        firstInfo = bs.find('div', {'class':'c-product__info'}).find('h1', {'class':'c-product__name'}).text.strip()
        duomenys['firstInfo'] = firstInfo

        secondInfo = bs.find('div', {'class':'c-accordion-item__title'}).text.replace('Prekės aprašymas:', '').strip()
        duomenys['secondInfo'] = secondInfo

        # kaina = bs.find('div', {'class':'c-price h-price--xx-large h-price--new'}).text.strip()
        # print(kaina)

        kaina = bs.find('div', {'class':'c-product__media'})
        price = eval(kaina['widget-data'].replace('null', 'None').replace('false', 'False').replace('true', 'True'))
        kaina2 = price['dataLayerItem'].get('price', 'nenurodyta')
        duomenys['kaina'] = kaina2

        specifikacija_keys = []
        specifikacija_values = []
        specifikacija = bs.find('div', {'class':'h-row h-pl--8 h-pr--8'}).find_all('tr')
        for trow in specifikacija:
            td = trow.find_all('td')
            key = td[0].text.strip()
            value = td[1].text.strip()
            specifikacija_keys.append(key)
            specifikacija_values.append(value)

        for k, v in zip(specifikacija_keys, specifikacija_values):
            duomenys[k] = v
            
        if kaina2 == 'nenurodyta':
            continue
        else:
            cols_i_need = list(duomenys.keys())
        
            data = tuple(duomenys.values())
            
            cols_i_have = getColNamesList(table='DviraciaiPigu', dbname='VarlePigu.db')
            
            alterTableAddColumn(cols_i_need, cols_i_have, 'DviraciaiPigu', 'VarlePigu.db')
            
            writeMany(data, 'DviraciaiPigu', 'VarlePigu.db')
            print(f'Įrašytas įrašas Nr {counts}, puslapis {psl}')
            counts = counts +1
            
    print(f'Įrašytas puslapis {psl}')
    psl = psl +1
    # if psl < iki_kokio_psl:
    if psl <= last_page:
        next_nuoroda = f'https://pigu.lt/lt/dviraciai?page={psl}'
        driver.get(next_nuoroda)
        time.sleep(5)
        print(f'kitas puslapis: {psl}')
        
driver.quit()
print('Baigta!!!')


34 https://pigu.lt/lt/dviraciai/vaikiskas-dviratis-16-vintage-beige-toimsa-16231?id=77231054
Įrašytas įrašas Nr 1980, puslapis 34
34 https://pigu.lt/lt/dviraciai/vaikiskas-dviratis-12-huffy-avengers-melynas?id=77231064
Įrašytas įrašas Nr 1981, puslapis 34
34 https://pigu.lt/lt/dviraciai/vaikiskas-dviratis-polar-jr-icecream-18-ivairiu?id=77614734
Įrašytas įrašas Nr 1982, puslapis 34
34 https://pigu.lt/lt/dviraciai/vaikiskas-dviratis-huffy-marvel-spider-man-72169w?id=78159743
Įrašytas įrašas Nr 1983, puslapis 34
34 https://pigu.lt/lt/dviraciai/miesto-dviratis-romet-orkan-21-pilkas?id=79398658
Įrašytas įrašas Nr 1984, puslapis 34
34 https://pigu.lt/lt/dviraciai/trekingo-dviratis-romet-gazela-0-2024-20l?id=79732355
Įrašytas įrašas Nr 1985, puslapis 34
34 https://pigu.lt/lt/dviraciai/miesto-dviratis-pegas-26-melynas?id=80155655
Įrašytas įrašas Nr 1986, puslapis 34
34 https://pigu.lt/lt/dviraciai/miesto-dviratis-pegasus-26-raudonas?id=80156215
Įrašytas įrašas Nr 1987, puslapis 34
34 https://